In [1]:
import pandas as pd
login_data = pd.read_csv('login_data.txt', sep='\t', parse_dates=['ts'])
login_data.head()

,id,uid,ts
0,1,466,2017-01-07 18:24:55
1,2,458,2017-01-07 18:25:18
2,3,458,2017-01-07 18:26:21
3,4,592,2017-01-07 19:09:59
4,5,393,2017-01-08 00:41:19


In [2]:
login_data.dtypes

id              int64
uid             int64
ts     datetime64[ns]
dtype: object

#### DAU计算

In [3]:
login_data['day'] = login_data['ts'].map(lambda x: x.strftime('%Y-%m-%d'))
uid_count = login_data.groupby('day').aggregate({'uid': lambda x: x.nunique()})
uid_count.reset_index(inplace=True)
uid_count

,day,uid
0,2017-01-07,3
1,2017-01-08,3
2,2017-01-09,9
3,2017-01-10,9
4,2017-01-11,8
5,2017-01-12,10
6,2017-01-13,8
7,2017-01-14,4
8,2017-01-15,4
9,2017-01-16,11


#### 留存计算

#### 次日留存计算

In [4]:
import pandas as pd
from datetime import timedelta

In [5]:
login_data = pd.read_csv('login_data.txt', sep='\t', parse_dates=['ts'])
login_data['day'] = login_data['ts'].map(lambda x: x.strftime('%Y-%m-%d'))
login_data['dt_ts'] = pd.to_datetime(login_data['day'], format='%Y-%m-%d')
login_data.head()

,id,uid,ts,day,dt_ts
0,1,466,2017-01-07 18:24:55,2017-01-07,2017-01-07
1,2,458,2017-01-07 18:25:18,2017-01-07,2017-01-07
2,3,458,2017-01-07 18:26:21,2017-01-07,2017-01-07
3,4,592,2017-01-07 19:09:59,2017-01-07,2017-01-07
4,5,393,2017-01-08 00:41:19,2017-01-08,2017-01-08


In [6]:
login_data.dtypes
#day是字符串格式
#dt_ts是datetime64格式,便于进行日期加减

id                int64
uid               int64
ts       datetime64[ns]
day              object
dt_ts    datetime64[ns]
dtype: object

In [7]:
data_1 = login_data.copy()
data_1['dt_ts_1'] = data_1['dt_ts'] + timedelta(-1)
data_1.head()

,id,uid,ts,day,dt_ts,dt_ts_1
0,1,466,2017-01-07 18:24:55,2017-01-07,2017-01-07,2017-01-06
1,2,458,2017-01-07 18:25:18,2017-01-07,2017-01-07,2017-01-06
2,3,458,2017-01-07 18:26:21,2017-01-07,2017-01-07,2017-01-06
3,4,592,2017-01-07 19:09:59,2017-01-07,2017-01-07,2017-01-06
4,5,393,2017-01-08 00:41:19,2017-01-08,2017-01-08,2017-01-07


In [8]:
#dt_ts_1是当前日期减一天，使用uid和dt_ts 关联，左边是第一天活跃的用户，右边是第二天活跃的用户
merge_1 = pd.merge(login_data, data_1, left_on=['uid', 'dt_ts'], right_on=['uid', 'dt_ts_1'], how='left')
merge_1.head(10)

,id_x,uid,ts_x,day_x,dt_ts_x,id_y,ts_y,day_y,dt_ts_y,dt_ts_1
0,1,466,2017-01-07 18:24:55,2017-01-07,2017-01-07,NaN,NaT,NaN,NaT,NaT
1,2,458,2017-01-07 18:25:18,2017-01-07,2017-01-07,NaN,NaT,NaN,NaT,NaT
2,3,458,2017-01-07 18:26:21,2017-01-07,2017-01-07,NaN,NaT,NaN,NaT,NaT
3,4,592,2017-01-07 19:09:59,2017-01-07,2017-01-07,6.0,2017-01-08 09:14:27,2017-01-08,2017-01-08,2017-01-07
4,4,592,2017-01-07 19:09:59,2017-01-07,2017-01-07,7.0,2017-01-08 11:18:08,2017-01-08,2017-01-08,2017-01-07
5,5,393,2017-01-08 00:41:19,2017-01-08,2017-01-08,19.0,2017-01-09 16:22:48,2017-01-09,2017-01-09,2017-01-08
6,6,592,2017-01-08 09:14:27,2017-01-08,2017-01-08,13.0,2017-01-09 12:47:22,2017-01-09,2017-01-09,2017-01-08
7,6,592,2017-01-08 09:14:27,2017-01-08,2017-01-08,20.0,2017-01-09 16:31:07,2017-01-09,2017-01-09,2017-01-08
8,6,592,2017-01-08 09:14:27,2017-01-08,2017-01-08,21.0,2017-01-09 17:03:37,2017-01-09,2017-01-09,2017-01-08
9,6,592,2017-01-08 09:14:27,2017-01-08,2017-01-08,22.0,2017-01-09 20:06:54,2017-01-09,2017-01-09,2017-01-08


In [11]:
init_user = merge_1.groupby('day_x').aggregate({'uid': lambda x: x.nunique()})
init_user.reset_index(inplace=True)
init_user.head()

,day_x,uid
0,2017-01-07,3
1,2017-01-08,3
2,2017-01-09,9
3,2017-01-10,9
4,2017-01-11,8


In [15]:
one_day_remain_user = merge_1[merge_1['day_y'].notnull()].groupby('day_x').aggregate({'uid': lambda x: x.nunique()})
one_day_remain_user.reset_index(inplace=True)
one_day_remain_user.head()

,day_x,uid
0,2017-01-07,1
1,2017-01-08,2
2,2017-01-09,5
3,2017-01-10,4
4,2017-01-11,6


In [17]:
merge_one_day = pd.merge(init_user, one_day_remain_user, on=['day_x'])
merge_one_day.head(20)

,day_x,uid_x,uid_y
0,2017-01-07,3,1
1,2017-01-08,3,2
2,2017-01-09,9,5
3,2017-01-10,9,4
4,2017-01-11,8,6
5,2017-01-12,10,5
6,2017-01-13,8,2
7,2017-01-14,4,2
8,2017-01-15,4,3
9,2017-01-16,11,5


In [20]:
merge_one_day['one_remain_rate'] = merge_one_day['uid_y'] / merge_one_day['uid_x']
merge_one_day['one_remain_rate'] = merge_one_day['one_remain_rate'].apply(lambda x: format(x, '.2%'))
merge_one_day.head(20)

,day_x,uid_x,uid_y,one_remain_rate
0,2017-01-07,3,1,33.33%
1,2017-01-08,3,2,66.67%
2,2017-01-09,9,5,55.56%
3,2017-01-10,9,4,44.44%
4,2017-01-11,8,6,75.00%
5,2017-01-12,10,5,50.00%
6,2017-01-13,8,2,25.00%
7,2017-01-14,4,2,50.00%
8,2017-01-15,4,3,75.00%
9,2017-01-16,11,5,45.45%


#### 多日留存计算 方法一

In [21]:
merge_all = pd.merge(login_data, login_data, on=['uid'], how='left')
merge_all['diff'] = (merge_all['dt_ts_y'] - merge_all['dt_ts_x']).map(lambda x: x.days)#使用map取的具体数字
merge_all.head()

,id_x,uid,ts_x,day_x,dt_ts_x,id_y,ts_y,day_y,dt_ts_y,diff
0,1,466,2017-01-07 18:24:55,2017-01-07,2017-01-07,1,2017-01-07 18:24:55,2017-01-07,2017-01-07,0
1,1,466,2017-01-07 18:24:55,2017-01-07,2017-01-07,42,2017-01-10 19:12:47,2017-01-10,2017-01-10,3
2,1,466,2017-01-07 18:24:55,2017-01-07,2017-01-07,111,2017-01-16 15:21:20,2017-01-16,2017-01-16,9
3,1,466,2017-01-07 18:24:55,2017-01-07,2017-01-07,120,2017-01-17 14:06:57,2017-01-17,2017-01-17,10
4,1,466,2017-01-07 18:24:55,2017-01-07,2017-01-07,125,2017-01-17 17:47:23,2017-01-17,2017-01-17,10


In [25]:
diff_0 = merge_all[merge_all['diff'] == 0].groupby('day_x')['uid'].nunique()
diff_1 = merge_all[merge_all['diff'] == 1].groupby('day_x')['uid'].nunique()
diff_6 = merge_all[merge_all['diff'] == 6].groupby('day_x')['uid'].nunique()
diff_13 = merge_all[merge_all['diff'] == 13].groupby('day_x')['uid'].nunique()
diff_0 = diff_0.reset_index()
diff_1 = diff_1.reset_index()
diff_6 = diff_6.reset_index()
diff_13 = diff_13.reset_index()

In [26]:
diff_0

,day_x,uid
0,2017-01-07,3
1,2017-01-08,3
2,2017-01-09,9
3,2017-01-10,9
4,2017-01-11,8
5,2017-01-12,10
6,2017-01-13,8
7,2017-01-14,4
8,2017-01-15,4
9,2017-01-16,11


In [42]:
liucun = pd.merge(pd.merge(pd.merge(diff_0, diff_1, on=['day_x'], how='left'), diff_6, on=['day_x'], how='left'), diff_13, on=['day_x'], how='left')
liucun.head()

,day_x,uid_x,uid_y,uid_x,uid_y
0,2017-01-07,3,1.0,2.0,2.0
1,2017-01-08,3,2.0,1.0,1.0
2,2017-01-09,9,5.0,3.0,NaN
3,2017-01-10,9,4.0,5.0,4.0
4,2017-01-11,8,6.0,5.0,1.0


In [45]:
liucun.columns=['day', 'init', 'one_day_remain', 'seven_day_remain', 'fifteen_day_remain']
liucun.fillna(0, inplace=True)
liucun.head(20)

,day,init,one_day_remain,seven_day_remain,fifteen_day_remain
0,2017-01-07,3,1.0,2.0,2.0
1,2017-01-08,3,2.0,1.0,1.0
2,2017-01-09,9,5.0,3.0,0.0
3,2017-01-10,9,4.0,5.0,4.0
4,2017-01-11,8,6.0,5.0,1.0
5,2017-01-12,10,5.0,7.0,3.0
6,2017-01-13,8,2.0,4.0,1.0
7,2017-01-14,4,2.0,3.0,2.0
8,2017-01-15,4,3.0,2.0,1.0
9,2017-01-16,11,5.0,1.0,0.0


#### 多日留存计算 方法二

In [32]:
login_data.head()

,id,uid,ts,day,dt_ts
0,1,466,2017-01-07 18:24:55,2017-01-07,2017-01-07
1,2,458,2017-01-07 18:25:18,2017-01-07,2017-01-07
2,3,458,2017-01-07 18:26:21,2017-01-07,2017-01-07
3,4,592,2017-01-07 19:09:59,2017-01-07,2017-01-07
4,5,393,2017-01-08 00:41:19,2017-01-08,2017-01-08


In [38]:
def cal_n_day_remain(df, n):
    dates = pd.Series(login_data.dt_ts.unique()).sort_values()[:-n]#取截止到n天的日期，保证有n日留存
    users = [] #定义列表存放初始用户数
    remains = []#定义列表存放留存用户数
    for d in dates:
        user = login_data[login_data['dt_ts'] == d]['uid'].unique()#当日活跃用户
        user_n_day = login_data[login_data['dt_ts']==d+timedelta(n)]['uid'].unique()#n日后活跃用户
        remain = [x for x in user_n_day if x in user]#取交际
        users.append(len(user))
        remains.append(len(remain))
    #一次循环计算一天的n日留存 
    #循环结束后构造dataframe并返回
    remain_df = pd.DataFrame({'days': dates, 'user': users, 'remain': remains})
    return remain_df

In [39]:
cal_n_day_remain(login_data, 1).head(20)

,days,user,remain
0,2017-01-07,3,1
1,2017-01-08,3,2
2,2017-01-09,9,5
3,2017-01-10,9,4
4,2017-01-11,8,6
5,2017-01-12,10,5
6,2017-01-13,8,2
7,2017-01-14,4,2
8,2017-01-15,4,3
9,2017-01-16,11,5


In [40]:
cal_n_day_remain(login_data, 6).head(20)

,days,user,remain
0,2017-01-07,3,2
1,2017-01-08,3,1
2,2017-01-09,9,3
3,2017-01-10,9,5
4,2017-01-11,8,5
5,2017-01-12,10,7
6,2017-01-13,8,4
7,2017-01-14,4,3
8,2017-01-15,4,2
9,2017-01-16,11,1


In [41]:
cal_n_day_remain(login_data, 13).head(20)

,days,user,remain
0,2017-01-07,3,2
1,2017-01-08,3,1
2,2017-01-09,9,0
3,2017-01-10,9,4
4,2017-01-11,8,1
5,2017-01-12,10,3
6,2017-01-13,8,1
7,2017-01-14,4,2
8,2017-01-15,4,1
9,2017-01-16,11,0


In [46]:
one_day_remain = cal_n_day_remain(login_data, 1)
seven_day_remain = cal_n_day_remain(login_data, 6)
fifteen_day_remain = cal_n_day_remain(login_data, 13)

In [47]:
liucun2 = pd.merge(pd.merge(one_day_remain, seven_day_remain[['days', 'remain']], on=['days'], how='left'), fifteen_day_remain[['days', 'remain']], on=['days'], how='left')
liucun2.head(20)

,days,user,remain_x,remain_y,remain
0,2017-01-07,3,1,2.0,2.0
1,2017-01-08,3,2,1.0,1.0
2,2017-01-09,9,5,3.0,0.0
3,2017-01-10,9,4,5.0,4.0
4,2017-01-11,8,6,5.0,1.0
5,2017-01-12,10,5,7.0,3.0
6,2017-01-13,8,2,4.0,1.0
7,2017-01-14,4,2,3.0,2.0
8,2017-01-15,4,3,2.0,1.0
9,2017-01-16,11,5,1.0,0.0
